In [3]:
# import os
# 
# import pandas as pd
# import rouge
# import numpy as np
# from tqdm import tqdm
# tqdm.pandas()
# from rouge import Rouge
# train = pd.read_csv('public_dat/train.csv')
# test = pd.read_csv('testfinal.csv')

In [5]:
import os

import pandas as pd
import rouge
import numpy as np
from tqdm import tqdm
tqdm.pandas()
train = pd.read_csv('public_dat/train.csv')
test = pd.read_csv('testfinal.csv')
import nltk
#apply stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
ps = PorterStemmer()
def stem(text):
    words = word_tokenize(text)
    return ' '.join([ps.stem(w) for w in words])
def preprocess(text):
    text = text.replace('\n', ' ')
    #remove number 
    text = ''.join([i for i in text if not i.isdigit()])
    #remove 1 and 2 length words
    text = ' '.join([w for w in text.split() if len(w) > 2])
    text = stem(text)

    if len(text)==0:
        
        return '[NUM]'
    
    return text
train['clean_description'] = train['description'].progress_apply(preprocess)
test['clean_description'] = test['description'].progress_apply(preprocess)

[nltk_data] Downloading package punkt_tab to C:\Users\Sagar
[nltk_data]     Rathore\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
100%|██████████| 184664/184664 [00:13<00:00, 13250.43it/s]


In [ ]:
def custom_simon_similarity(pairs1, pairs2):
    """
    Perform trigram comparison between two small input strings
    and return a percentage match in decimal form.
    Optimized for small inputs by reducing redundant loops.
    """
    # Convert pairs2 to a set for O(1) lookup times
    pairs2_set = set(pairs2)

    hit_count = sum(1 for x in pairs1 if x in pairs2_set)

    union = len(pairs1) + len(pairs2)
    
    if union == 0:
        return 0

    return 0.6 * hit_count / (min(len(pairs1), len(pairs2)) + 1) + 0.4 * hit_count / (len(pairs1) + 1)
        # Convert pairs2 to a set for O(1) lookup times

def get_trigrams(string):
    """
    Take a string and return a list of bigrams.
    """
    if string is None:
        return ""

    s = string.lower()
    return [s[i : i + 3] for i in list(range(len(s) - 1))]


In [ ]:
train['bigram'] = train['clean_description'].progress_apply(get_trigrams)
test['bigram'] = test['clean_description'].progress_apply(get_trigrams)

100%|██████████| 184664/184664 [00:00<00:00, 329712.96it/s]


In [ ]:
def match_simon(test_row, train):
    scores = train['bigram'].apply(lambda x: custom_simon_similarity(test_row, x))
    return scores


In [ ]:
#find for each row in test
#apply tqdm for loop

# matching simon similarity for each row in test with all test rows and storing top 30 most matching instances
for i in tqdm(range(test.shape[0])):
    score = match_simon(test['bigram'][i], train)
    top30 = train.iloc[score.sort_values(ascending=False).head(30).index]
    test.loc[i, 'supergroup'] = top30['supergroup'].value_counts().idxmax()
    test.loc[i, 'group'] = top30['group'].value_counts().idxmax()
    test.loc[i, 'module'] = top30['module'].value_counts().idxmax()
    # save the top 30 matching instances
    
test.to_csv('submissiontest.csv', index=False)


In [ ]:
# Prediction
#for every row in train
submission_test = pd.read_csv('submissiontest.csv')
# t5 predictions brand
testt5 = pd.read_csv("testpredictt51st.csv")

sub = pd.DataFrame()
for index, row in tqdm(submission_test.iterrows(), total=submission_test.shape[0]):
    indexes = row['si_index']
    #select these indexes from the tr (training dataset) dataframe
    selected = train.iloc[indexes]
    #selected supergroup with max count in top 30
    supergroup = row['supergroup']
    selected = selected[selected['supergroup'] == supergroup]
    score = match_simon(submission_test['bigram'][index], selected)
    selected = selected[score == score.max()]
    # select group with max simon similarity in that selected supergroup from top30
    # select module with max simon similarity in that selected group from top30
    group = selected['group'].value_counts().idxmax()
    module = selected['module'].value_counts().idxmax()
    sub.at[index, 'indoml_id'] = row['indoml_id']
    sub.at[index, 'supergroup'] = supergroup
    sub.at[index, 'group'] = group
    sub.at[index, 'module'] = module

sub['brand'] = testt5['brand']

import json
categories = ['supergroup','group','module','brand']

with open('submitpredictions.predict', 'w') as file:
    for index, row in sub.iterrows():
        result = {}
        result['indoml_id'] = row['indoml_id']
        for category in categories:
            result[category] = row[category]
        file.write(json.dumps(result) + '\n')